In [1]:
import pandas as pd
from osmapi import OsmApi
MyApi = OsmApi()
import pyproj
#import polars as pl
pd.set_option('display.max_columns', 500)

pd.options.mode.chained_assignment = None  # default='warn'

geodesic = pyproj.Geod(ellps='WGS84')



In [2]:
#     geodesic = pyproj.Geod(ellps='WGS84')
#     fwd_azimuth,back_azimuth,distance = geodesic.inv(x, y, row['data.lon'], row['data.lat'])
    
    
#     p1_lat = 40.484215
#     p1_lon = -74.662637
    
#     p2_lat = 40.484217 
#     p2_lon = -74.662071
    
#     #p3 is north east
#     p3_lat = 40.484546
#     p3_lon = -74.661981
    
    
#     print( geodesic.inv(p1_lon, p1_lat, p3_lon, p3_lat))

In [3]:
inventoryDf = pd.read_excel('Crossing_CA.xlsx')

In [4]:
inventoryDf.columns

Index(['revisiondate', 'reportingagencytypecode', 'reporting_agency_type',
       'reasoncode', 'reasondescription', 'crossingid', 'crossingidsuffix',
       'reportingagencycode', 'reportingagencyname', 'statecode',
       ...
       'Railroad/Company Grouping', 'SMT Grouping',
       'Parent Railroad/Company Code', 'Parent Railroad/Company Name',
       'Holding Company', 'Principal Warning Device', 'Other Track',
       'Multiple Forms Filed', 'Other Track Description', 'Geocoded Lat/Long'],
      dtype='object', length=255)

In [5]:
#Cast Date Column of InventoryDf

inventoryDf['revisiondate'] = pd.to_datetime(inventoryDf['revisiondate'])

#Sort and Drop duplicates


inventoryDf = inventoryDf.sort_values('revisiondate').drop_duplicates('crossingid',keep='last')
#df.sort_values('DATE_CHANGED').drop_duplicates('STATION_ID',keep='last')


In [6]:
inventoryDf.shape

(18394, 255)

In [7]:
inventoryDf = inventoryDf[inventoryDf['latitude'].notnull()]

In [8]:
inventoryDf.shape

(17120, 255)

In [9]:
inventoryDf = inventoryDf[inventoryDf['longitude'].notnull()]

In [10]:
inventoryDf.shape

(17120, 255)

In [11]:
inventorySubset = inventoryDf.head(10)

In [12]:
from collections.abc import MutableMapping

def flatten_dict(d: MutableMapping, sep: str= '.') -> MutableMapping:
    [flat_dict] = pd.json_normalize(d, sep=sep).to_dict(orient='records')
    return flat_dict

In [13]:
inventoryDf.head(10)

,revisiondate,reportingagencytypecode,reporting_agency_type,reasoncode,reasondescription,crossingid,crossingidsuffix,reportingagencycode,reportingagencyname,statecode,statename,countycode,countyname,innearcode,innear,citycode,cityname,citydescription,street,blocknumber,highwayname,separatetrack,separatetrackrailroad1,separatetrackrailroad2,separatetrackrailroad3,separatetrackrailroad4,sametrack,sametrackrailroad1,sametrackrailroad2,sametrackrailroad3,sametrackrailroad4,railroaddivision,railroadsubdivision,branchname,railroadmilepostprefix,railroadmilepostnumber,railroadmilepostsuffix,linesegment,nearesttimetablestation,timetablestation,parentrailroadcode,crossingownercode,crossingtypecode,crossingtype,crossingpurposecode,crossingpurpose,crossingpositioncode,crossingposition,publicaccess,typeoftrainserviceids,typeoftrainserviceid1,typeoftrainservice1,typeoftrainserviceid2,typeoftrainservice2,typeoftrainserviceid3,typeoftrainservice3,typeoftrainserviceid4,typeoftrainservice4,typeoftrainserviceid5,typeoftrainservice5,typeoftrainserviceid6,typeoftrainservice6,lessthanonepassengertrai,numberpassengertrainperday,developmenttypecode,development_type,adjacentcrossing,adjacentcrossingnumber,whistlebancode,whistleban,whistledate,hsrcorridoridsuffix,hsrcorridorid,latitude,longitude,latlongsourcecode,latlongsource,railroadnarrative,railroad_narrative_base,statenarrative,state_narrative_base,emergencytelephonenumber,railroadcontacttelephonenumber,statecontacttelephonenumber,totaldaylightthrutrains,totalnighttimethrutrains,totalswitchingtrains,totaltransittrains,movementsperdaycode,movementsperday,trainsperweek,trainsperyear,maximumtimetablespeed,minimumspeedrangeovercrossing,maximumspeedrangeovercrossing,numberofmaintracks,numberofsidingtracks,numberofyardtracks,numberoftransittracks,numberofindistrytracks,traindetectionids,traindetectionid1,traindetection1,traindetectionid2,traindetection2,traindetectionid3,traindetection3,tracksignaled,eventrecorder,remotehealthmonitoring,signsorsignals,numbercrossbuckassemblies,numberstopsigns,numberyieldsigns,advancewarnings,advancewarning1,advancewarningtext1,advancewarning2,advancewarningtext2,advancewarning3,advancewarningtext3,advancewarning4,advancewarningtext4,advancewarning5,advancewarningtext5,advancewarning6,advancewarningtext6,advancewarning7,advancewarningtext7,countadvw10_1,countadvw10_2,countadvw10_3,countadvw10_4,countadvw10_11,countadvw10_12,lowgroundclearancesigns,numberlowgroundclearancesigns,pavementmarkingids,pavementmarkingid1,pavementmarking,pavementmarkingid2,pavementmarking2,pavementmarkingid3,pavementmarking3,pavementmarkingid4,pavementmarking4,channeldevicemediancode,channeldevicemedian,exemptsigns,enssigndisplayed,othermutcodesigns,numberothermutcodesigns1,othermutcodesignstype1,other_mutcd_signs_type_text_1,numberothermutcodesigns2,othermutcodesignstype2,other_mutcd_signs_type_text_2,numberothermutcodesigns3,othermutcodesignstype3,other_mutcd_signs_type_text_3,privatecrossingsigns,ledenhancedsigns,countroadwaygatearms,countpedestriangatearms,gateconfigurationcode,gateconfiguration,gateconftype,gate_configuration_type_description,countcantileveredstructu,countcantileveredstructu_1,cantileveredtypestructurecode,cantileveredtypestructure,countmastmountedflashinglight,mastmountedflashinglight,mastmountedflashinglighttype,mastmountedflashingblacklight,mastmountedflashingsidelight,countflashinglightpair,installationdateactivewa,waysidehorn,installationdatewaysidehorn,highwaytrafficsignal,numberofbells,nontrainactivewarningcode,nontrainactivewarning,countotherflashinglights,otherflashinglightstype,nearbyhighwaytrafficsignals,highwaytrafficsignalinte,highwaytrafficsignalinte_1,highwaytrafficsignalpree,highwaytrafficsignalpreemption,highwaytrafficpresignals,stoplinedistance,storagedistance,highwaymonitoringdevicescode,highwaymonitoringdevices,trafficlane,trafficlanetypecode,trafficlanetypedescription,highwaypaved,trackrundownstreet,crossingilluminated,crossingsurfaceinstallat,crossing

In [14]:
allRows = []

count = 0

for i, row in inventoryDf.iloc[1000:].iterrows():
    
    print(count, ' FRA: ', row['crossingid'])
    count = count + 1
    
    try:
    
        fra_crossing_id = row['crossingid']
        fra_lat = row['latitude']
        fra_lon = row['longitude']
        
        

        search_level = 0.001
        max_search_level = 0.005
        
        results_dict = MyApi.Map(fra_lon-search_level,fra_lat-search_level,fra_lon+search_level,fra_lat+search_level)
        
        flat_dict = []
        for d in results_dict:
            flat_dict.append(flatten_dict(d))

        df = pd.DataFrame.from_dict(flat_dict)
        
        crossings_df = pd.DataFrame()
        
        if('data.tag.railway' in df.columns):
            crossings_df= df[df['data.tag.railway'] == 'level_crossing']
         
        
        while search_level < max_search_level and crossings_df.shape[0] <= 0:
            search_level = search_level* 2
            print('search area increased to ', search_level)
            results_dict = MyApi.Map(fra_lon-search_level,fra_lat-search_level,fra_lon+search_level,fra_lat+search_level)
        
            flat_dict = []
            for d in results_dict:
                flat_dict.append(flatten_dict(d))

            df = pd.DataFrame.from_dict(flat_dict)
            
            if('data.tag.railway' in df.columns):
                crossings_df= df[df['data.tag.railway'] == 'level_crossing']
            
            
        if('data.tag.railway' not in df.columns):
            print("still missing rail")
            continue
        
        if(crossings_df.shape[0] <= 0):
            print("still missing crossing")
            continue


       


        crossings_df['customdistance']= crossings_df.apply(lambda g: geodesic.inv(fra_lon, fra_lat, g['data.lon'], g['data.lat'])[2],axis=1)
        crossings_df = crossings_df.sort_values(by=['customdistance'])

        crossing_node = crossings_df.iloc[0]['data.id']

        osm_crossing_lat = crossings_df.iloc[0]['data.lat']
        osm_crossing_lon = crossings_df.iloc[0]['data.lon']

        ways = df[df['data.nd'].notnull()]
        inWays = ways[ways['data.nd'].apply(lambda x: crossing_node in x)]


        inWays['prev_node'] = inWays['data.nd'].apply(lambda x: x[x.index(crossing_node)-1])

        #get data on the previous way
        inWays['prev_node_lat'] = inWays.apply(lambda x: MyApi.NodeGet(x['prev_node'])['lat'],axis=1)
        inWays['prev_node_lon'] = inWays.apply(lambda x: MyApi.NodeGet(x['prev_node'])['lon'],axis=1)


        inWays['fwd_azimuth_to_crossing']= inWays.apply(lambda g: geodesic.inv(osm_crossing_lon, osm_crossing_lat, g['prev_node_lon'], g['prev_node_lat'])[0],axis=1)
        inWays['back_azimuth_to_crossing']= inWays.apply(lambda g: geodesic.inv(osm_crossing_lon, osm_crossing_lat, g['prev_node_lon'], g['prev_node_lat'])[1],axis=1)
        inWays['distance_to_crossing']= inWays.apply(lambda g: geodesic.inv(osm_crossing_lon, osm_crossing_lat, g['prev_node_lon'], g['prev_node_lat'])[2],axis=1)

        inWays = inWays.assign(related__OSM_Node=crossing_node)


        inWays = inWays.assign(related__OSM_Node_lat=osm_crossing_lat)
        inWays = inWays.assign(related__OSM_Node_lon=osm_crossing_lon)

        inWays = inWays.assign(realted_FRA_ID=fra_crossing_id)

        
        
        for iS, rowS in inWays.iterrows():
            allRows.append(rowS.to_dict())
        
           # print(inWays.to_dict())

            
    except Exception  as e:
        print('something went wrong with :' , i,row['crossingid'], e)
    
    

    
df = pd.DataFrame(allRows)    

0  FRA:  029324M
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
1  FRA:  029168D
2  FRA:  029425Y
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
3  FRA:  029329W
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
4  FRA:  029135R
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
5  FRA:  029124D
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
6  FRA:  029307W
7  FRA:  029386K
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
8  FRA:  029183F
9  FRA:  029466D
search area increased to  0.002
search area increased to  0.004
10  FRA:  029330R
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still m

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100  FRA:  029279V
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
101  FRA:  029276A
102  FRA:  029438A
103  FRA:  029315N
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
104  FRA:  029420P
search area increased to  0.002
105  FRA:  029299G
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
106  FRA:  029136X
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
107  FRA:  029328P
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
108  FRA:  026115U
109  FRA:  029388Y
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
110  FRA:  029295E
111  FRA:  029419V
112  FRA:  029300Y
search area increased to  0.002
search area increased to  0.004
search area 

search area increased to  0.008
still missing crossing
198  FRA:  768342Y
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
199  FRA:  768347H
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
200  FRA:  768332T
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
201  FRA:  768297G
202  FRA:  768348P
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
203  FRA:  768321F
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
204  FRA:  768306D
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
205  FRA:  768327W
search area increased to  0.002
search area increased to  0.004
search area inc

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


...

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


search area increased to  0.004
15615  FRA:  027992J
search area increased to  0.002
search area increased to  0.004
15616  FRA:  748137N
15617  FRA:  761606T
15618  FRA:  027505K
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
15619  FRA:  835116J
15620  FRA:  661908U
15621  FRA:  028012B
15622  FRA:  028002V
search area increased to  0.002
15623  FRA:  766163B
15624  FRA:  028004J
15625  FRA:  028010M
search area increased to  0.002
15626  FRA:  028142X
15627  FRA:  028009T
search area increased to  0.002
search area increased to  0.004
15628  FRA:  028018S
15629  FRA:  746930V
15630  FRA:  746932J
15631  FRA:  028003C
15632  FRA:  028008L
search area increased to  0.002
15633  FRA:  753401E
15634  FRA:  747697S
search area increased to  0.002
15635  FRA:  028020T
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
15636  FRA:  028001N
15637  FRA:  028011U
15638  FRA:  0

search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
15843  FRA:  026604E
15844  FRA:  026282T
15845  FRA:  026609N
15846  FRA:  026602R
15847  FRA:  967129A
15848  FRA:  026276P
15849  FRA:  943886C
15850  FRA:  026288J
15851  FRA:  026264V
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
15852  FRA:  026274B
15853  FRA:  026277W
15854  FRA:  026242V
search area increased to  0.002
15855  FRA:  026266J
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
15856  FRA:  026603X
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
15857  FRA:  026608G
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
15858  FRA:  026246X
search area increased to  0.002
search area increased to  0.

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


15959  FRA:  027842B
15960  FRA:  027983K
search area increased to  0.002
search area increased to  0.004
15961  FRA:  980231L
15962  FRA:  027852G
search area increased to  0.002
15963  FRA:  028078B
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
something went wrong with : 18307 028078B Request failed: 400 - Bad Request - b'You requested too many nodes (limit is 50000). Either request a smaller area, or use planet.osm'
15964  FRA:  027839T
15965  FRA:  027971R
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
15966  FRA:  746908H
15967  FRA:  027861F
15968  FRA:  027838L
15969  FRA:  761607A
15970  FRA:  027973E
search area increased to  0.002
search area increased to  0.004
search area increased to  0.008
still missing crossing
15971  FRA:  026318Y
search area increased to  0.002
15972  FRA:  027837E
15973  FRA:  811490J
15974  FRA:  927839D
15975  FRA:  761553W
sear

In [ ]:
df

In [15]:
import time  

time_stamp = time.time()

file_name = 'CA_AZ_DATA' + str(time_stamp) + '.csv'

df.to_csv(file_name)

In [ ]:
y = 38.8067190
x = -121.2280509


i = 0.001


x1 = x + i
y1 = y + i


x2 = x + i
y2 = y - i


x3 = x - i
y3 = y - i


x4 = x - i
y4 = y + i

results_dict = MyApi.Map(x3,y3,x1,y1)

In [ ]:
results_dict

In [ ]:
df

#for each crossing find distance to point

distances

for z, row  in df.iterrows():
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(x, y, row['data.lon'], row['data.lat'])
    print(z)
    distances.append(distance)
    zrow.append(z)
    print('fwd: ',back_azimuth)
    print(geodesic.inv(x, y, row['data.lon'], row['data.lat'])[2])
    


In [ ]:
df

In [ ]:

geodesic = pyproj.Geod(ellps='WGS84')

y = 38.8067190
x = -121.2280509

crossings_df['customdistance']= df.apply(lambda g: geodesic.inv(x, y, g['data.lon'], g['data.lat'])[2],axis=1)
crossings_df = crossings_df.sort_values(by=['customdistance'])
crossings_df

In [ ]:
#now we have the nearest level crossing node
#time to search for it contained in ways

#select level_crossing node
crossing_node = crossings_df.iloc[0]['data.id']

print(crossing_node)

ways = df[df['data.nd'].notnull()]
inWays = ways[ways['data.nd'].apply(lambda x: crossing_node in x)]
inWays

In [ ]:
#should return two
inWays.shape

In [ ]:
#find the previous way

inWays['prev_node'] = inWays['data.nd'].apply(lambda x: x[x.index(crossing_node)-2])

#get data on the previous way
inWays['prev_node_lat'] = inWays.apply(lambda x: MyApi.NodeGet(x['prev_node'])['lat'],axis=1)
inWays['prev_node_lon'] = inWays.apply(lambda x: MyApi.NodeGet(x['prev_node'])['lon'],axis=1)

#calculate heading from these nodes to central node's 
y = 38.8067190
x = -121.2280509


inWays['fwd_azimuth_to_crossing']= inWays.apply(lambda g: geodesic.inv(x, y, g['prev_node_lon'], g['prev_node_lat'])[0],axis=1)
inWays['back_azimuth_to_crossing']= inWays.apply(lambda g: geodesic.inv(x, y, g['prev_node_lon'], g['prev_node_lat'])[1],axis=1)
inWays['distance_to_crossing']= inWays.apply(lambda g: geodesic.inv(x, y, g['prev_node_lon'], g['prev_node_lat'])[2],axis=1)


inWays

In [ ]:
#get the rail data
#inWays[inWays['data.tag.railway'] == 'rail'].iloc[0]

inWays = inWays.assign(related_Node=crossing_node)
inWays

In [ ]:
#FRA Crossing ID, FRA Lon, FRA Lat, OSM Crossing Node, OSM Nodes Found, OSM Found Rail and Road, OSM Road Azimuth, OSM Rail Azimuth 

